Tests concerning the parameters characterising the magnestophere of the systems.

In [1]:
import os
import scipy.io as sio
from scipy.io import readsav
from scipy.optimize import curve_fit
import numpy as np
from math import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib widget
import Star 
import Planet 

In [2]:
from calc_tools import *
from DynamoRegion import *
from magneticmoment import *
from StellarWind import *

In [3]:
MS=1.989e30 #kg
RS=6.96342e8 #m
AS=4.6e9 #yo
BSsw= 1 #T
LS=3.826e26 #W


sol=Star('Sun', MS, RS, AS, BSsw,LS)

MJ=1.8986e27 #kg
#RJ=69911e3   #m
RJ=71492e3
wJ=1.77e-4   #s-1
LJ_norm=1.31e-9 #Ls
dJ=5.2*149597870700

jup=Planet('Jupiter', MJ, RJ, wJ,wJ,LJ_norm,dJ)


sol.affiche()
jup.affiche()

Name :  Sun
Mass :  1.989e+30  MS
Radius :  696342000.0  RS
Age :  4600000000.0  yS
SW magnetic field  1 BswS
Luminosity :  3.826e+26 LS
Name :  Jupiter
Mass :  1.8986e+27  kg
Radius :  71492000.0  m
Rotation rate :  0.000177  s-1
Orbital period :  0.000177 s-1
Luminosity :  1.31e-09 W
Distance to host star :  777908927640.0 m
Radius of the magnetosphere :  0.5 m


- Computing the parameters of the dynamo region and the values of the magnetic moment for different models

- - Dynamo region

In [4]:
test_targets=pd.DataFrame({'star_radius' : [1.0,1.0,1.18,1.12,1.48,1.48,1.48], 'star_mass':[1.0,1.0,1.06,1.1,1.42,1.42,1.42],'star_age' : [4.6,4.6,5.5,3.0,1.0,1.0,1.0], 'planet_radius':[1.0,0.84,1.42,1.25,1.2,1.58,1.48], 'planet_mass':[1.0,0.3, 0.69,1.18,4.4,7.0,10.0], 'planet_orbital_frequency':[1.0,0.93,0.12,0.34,0.12,0.12,0.12],'planet_luminosity':[1.31e-9,1.31e-9,1.05e-9,2.06e-9,6.0e-9,6.0e-9,4.09e-7], 'planet_distance':[5.2,9.5,0.045,0.0225,0.0489,0.0489,0.0489]}, index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
#print(test_targets)
#print(test_targets.loc['Jupiter'])

rc1=[]
rc2=[]
rhoc1=[]
rhoc2=[]

rc1_norm=[]
rc2_norm=[]

radius=[]

for x in test_targets.itertuples():
    pla=Planet(x.Index,x.planet_mass, x.planet_radius, x.planet_orbital_frequency,1,1, x.planet_distance)
    r1=LaneEmden(pla.mass*jup.mass, pla.radius*jup.radius,1000)
    r2=LaneEmden(pla.mass*jup.mass, pla.radius*jup.radius,700)
    rc1.append(r1)
    rc2.append(r2)
    rc1_norm.append(r1/(pla.radius*jup.radius))
    rc2_norm.append(r2/(pla.radius*jup.radius))

    rho1=rhodyn(pla.mass*jup.mass,pla.radius*jup.radius,r1)
    rho2=rhodyn(pla.mass*jup.mass,pla.radius*jup.radius,r2)
    rhoc1.append(rho1)
    rhoc2.append(rho2)
    radius.append(pla.radius*jup.radius)

rc1_normJ=np.array(rc1)/rc1[0]
rc2_normJ=np.array(rc2)/rc2[0]
rhoc1_norm=np.array(rhoc1)/rhoc1[0]
rhoc2_norm=np.array(rhoc2)/rhoc2[0]

test_results_dynamo=pd.DataFrame({'rc1':rc1_norm, 'rc2':rc2_norm, 'rhoc1':rhoc1_norm, 'rhoc2':rhoc2_norm}, index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'] )


print(test_results_dynamo)

                        rc1       rc2     rhoc1     rhoc2
Jupiter            0.791454  0.848782  1.000000  1.000000
Saturn             0.615022  0.721287  0.676447  0.647143
HD 209458b         0.000000  0.438998  0.000000  0.442728
OGLE-TR-56b        0.672431  0.762088  0.742792  0.718751
T-bootes (light)   0.911164  0.936492  1.957362  2.065509
T-bootes (medium)  0.875936  0.910804  1.483899  1.536665
T-bootes (heavy)   0.925786  0.947091  2.285726  2.433248


In [5]:
9.8e7/105808160.0

0.9262045573800736

- - Magnetic moments without tidal locking

In [6]:
table_1MJ=pd.read_csv(r'/Users/emauduit/Documents/Thèse/Sélection des cibles/Programmes/1MJ.csv', delimiter=';')
table_5MJ=pd.read_csv(r'/Users/emauduit/Documents/Thèse/Sélection des cibles/Programmes/5MJ.csv', delimiter=';')
table_10MJ=pd.read_csv(r'/Users/emauduit/Documents/Thèse/Sélection des cibles/Programmes/10MJ.csv', delimiter=';')


In [7]:
M_sano1=[]
M_sano2=[]
M_Busse1=[]
M_Busse2=[]
M_MizuM1=[]
M_MizuM2=[]
M_MizuS1=[]
M_MizuS2=[]
M_RC=[]


i=0
for x in test_targets.itertuples():
    r1=rc1_normJ[i]
    r2=rc2_normJ[i]
    rho1=rhoc1_norm[i]
    rho2=rhoc2_norm[i]
    
    star=Star('Name', x.star_mass, x.star_radius, x.star_age,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency,x.planet_orbital_frequency,x.planet_luminosity,x.planet_distance)
    
    M_sano1.append(sano(r1,pla.rotrate,rho1))
    M_sano2.append(sano(r2,pla.rotrate,rho2))
    M_Busse1.append(Busse(r1,pla.rotrate,rho1))
    M_Busse2.append(Busse(r2,pla.rotrate,rho2))
    M_MizuM1.append(Mizu_moderate(r1,pla.rotrate,rho1,1.0))
    M_MizuM2.append(Mizu_moderate(r2,pla.rotrate,rho2,1.0))
    M_MizuS1.append(Mizu_slow(r1,pla.rotrate,rho1,1.0))
    M_MizuS2.append(Mizu_slow(r2,pla.rotrate,rho2,1.0))

    M_RC.append(Reiners_Christensen(pla,star, jup,sol,table_1MJ,table_5MJ,table_10MJ))
    print(pla.luminosity)
    i+=1


test_results_moments=pd.DataFrame({'M_sano1': M_sano1, 'M_sano2' : M_sano2, 'M_RC' : M_RC, 'M_Busse1': M_Busse1, 'M_Busse2':M_Busse2, 'M_MizuM1':M_MizuM1, 'M_MizuM2':M_MizuM2, 'M_MizuS1':M_MizuS1,'M_MizuS2':M_MizuS2},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_moments) 


jup
dans RC : L= 1.0
1.31e-09
dans RC : L= 1.0
1.31e-09
dans RC : L= 0.8015267175572519
1.05e-09
dans RC : L= 1.5725190839694656
2.06e-09
dans RC : L= 4.580152671755725
6e-09
dans RC : L= 4.580152671755725
6e-09
dans RC : L= 312.21374045801525
4.09e-07
                    M_sano1   M_sano2       M_RC  M_Busse1  M_Busse2  \
Jupiter            1.000000  1.000000   1.000000  1.000000  1.000000   
Saturn             0.171872  0.229907   0.084579  0.138860  0.194244   
HD 209458b         0.000000  0.027108   1.243191  0.000000  0.023231   
OGLE-TR-56b        0.361721  0.431702   1.973624  0.372769  0.457345   
T-bootes (light)   0.520294  0.460582   4.614555  0.611540  0.529971   
T-bootes (medium)  1.033584  0.944000   8.631402  1.366777  1.229178   
T-bootes (heavy)   1.238529  1.083367  33.940906  1.629593  1.392209   

                   M_MizuM1  M_MizuM2  M_MizuS1  M_MizuS2  
Jupiter            1.000000  1.000000  1.000000  1.000000  
Saturn             0.175019  0.234116  0.220593  0

- - Magnetic moments with tidal locking

In [8]:
M_sano1=[]
M_sano2=[]
M_Busse1=[]
M_Busse2=[]
M_MizuM1=[]
M_MizuM2=[]
M_MizuS1=[]
M_MizuS2=[]
M_RC=[]


i=0
for x in test_targets.itertuples():
    r1=rc1_normJ[i]
    r2=rc2_normJ[i]
    rho1=rhoc1_norm[i]
    rho2=rhoc2_norm[i]
    
    star=Star('Name', x.star_mass, x.star_radius, x.star_age,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency, x.planet_orbital_frequency,x.planet_luminosity,x.planet_distance)
    pla.tidal_locking(star.age,star.mass)
    print(pla.rotrate, pla.orbitperiod)

    M_sano1.append(sano(r1,pla.rotrate,rho1))
    M_sano2.append(sano(r2,pla.rotrate,rho2))
    M_Busse1.append(Busse(r1,pla.rotrate,rho1))
    M_Busse2.append(Busse(r2,pla.rotrate,rho2))
    M_MizuM1.append(Mizu_moderate(r1,pla.rotrate,rho1,1.0))
    M_MizuM2.append(Mizu_moderate(r2,pla.rotrate,rho2,1.0))
    M_MizuS1.append(Mizu_slow(r1,pla.rotrate,rho1,1.0))
    M_MizuS2.append(Mizu_slow(r2,pla.rotrate,rho2,1.0))

    M_RC.append(Reiners_Christensen(pla,star, jup,sol,table_1MJ,table_5MJ,table_10MJ))
    i+=1


test_results_moments_TL=pd.DataFrame({'M_sano1': M_sano1, 'M_sano2' : M_sano2, 'M_RC' : M_RC, 'M_Busse1': M_Busse1, 'M_Busse2':M_Busse2, 'M_MizuM1':M_MizuM1, 'M_MizuM2':M_MizuM2, 'M_MizuS1':M_MizuS1,'M_MizuS2':M_MizuS2},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_moments_TL)   

1.0 1.0
jup
dans RC : L= 1.0
1.0 0.93
dans RC : L= 1.0
0.12 0.12
dans RC : L= 0.8015267175572519
0.34 0.34
dans RC : L= 1.5725190839694656
0.12 0.12
dans RC : L= 4.580152671755725
0.12 0.12
dans RC : L= 4.580152671755725
0.12 0.12
dans RC : L= 312.21374045801525
                    M_sano1   M_sano2       M_RC  M_Busse1  M_Busse2  \
Jupiter            1.000000  1.000000   1.000000  1.000000  1.000000   
Saturn             0.184809  0.247212   0.084579  0.149312  0.208865   
HD 209458b         0.000000  0.027108   1.243191  0.000000  0.023231   
OGLE-TR-56b        0.361721  0.431702   1.973624  0.372769  0.457345   
T-bootes (light)   0.520294  0.460582   4.614555  0.611540  0.529971   
T-bootes (medium)  1.033584  0.944000   8.631402  1.366777  1.229178   
T-bootes (heavy)   1.238529  1.083367  33.940906  1.629593  1.392209   

                   M_MizuM1  M_MizuM2  M_MizuS1  M_MizuS2  
Jupiter            1.000000  1.000000  1.000000  1.000000  
Saturn             0.184809  0.247212  0

- Solar wind parameters

In [9]:
veff_46=[]
Tcor_46=[]
n_46=[]

dua=1.49597870700e11 #m

i=0
for x in test_targets.itertuples():
    star=Star('Name', x.star_mass*1.989e30, x.star_radius, 4.6e9,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency, x.planet_orbital_frequency,1,x.planet_distance*dua)

    if (pla.stardist <= 0.1*dua):
        v,veff1,n1,T1=CME(star,pla)
    else:
        v,veff1,n1,T1=parker(pla,star)
    
    veff_46.append(veff1)
    Tcor_46.append(T1)
    n_46.append(n1)

    i+=1


test_results_SW_46=pd.DataFrame({'V_eff': veff_46, 'T_corona' : Tcor_46, 'n' : n_46},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_SW_46)   

                           V_eff      T_corona             n
Jupiter            523793.993939  8.124286e+05  1.977061e+05
Saturn             555223.685861  8.124286e+05  5.587309e+04
HD 209458b         545503.128080  2.000000e+06  7.553582e+10
OGLE-TR-56b        565728.298376  2.000000e+06  6.001124e+11
T-bootes (light)   549943.863979  2.000000e+06  5.891486e+10
T-bootes (medium)  549943.863979  2.000000e+06  5.891486e+10
T-bootes (heavy)   549943.863979  2.000000e+06  5.891486e+10


In [10]:
veff_1=[]
Tcor_1=[]
n_1=[]

dua=1.49597870700e11 #m

i=0
for x in test_targets.itertuples():
    star=Star('Name', x.star_mass*1.989e30, x.star_radius, 1e9,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency, x.planet_orbital_frequency,1,x.planet_distance*dua)

    if (pla.stardist <= 0.1*dua):
        v,veff1,n1,T1=CME(star,pla)
    else:
        v,veff1,n1,T1=parker(pla,star)
    
    veff_1.append(veff1)
    Tcor_1.append(T1)
    n_1.append(n1)

    i+=1


test_results_SW_1=pd.DataFrame({'V_eff': veff_1, 'T_corona' : Tcor_1, 'n' : n_1},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_SW_1)  

                           V_eff   T_corona             n
Jupiter            918147.119974  2091000.0  1.973051e+06
Saturn             964364.390346  2091000.0  5.627904e+05
HD 209458b         545503.128080  2000000.0  7.553582e+10
OGLE-TR-56b        565728.298376  2000000.0  6.001124e+11
T-bootes (light)   549943.863979  2000000.0  5.891486e+10
T-bootes (medium)  549943.863979  2000000.0  5.891486e+10
T-bootes (heavy)   549943.863979  2000000.0  5.891486e+10


In [11]:
veff_07=[]
Tcor_07=[]
n_07=[]

dua=1.49597870700e11 #m

i=0
for x in test_targets.itertuples():
    star=Star('Name', x.star_mass*1.989e30, x.star_radius, 0.7e9,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency, x.planet_orbital_frequency,1,x.planet_distance*dua)

    if (pla.stardist <= 0.1*dua):
        v,veff1,n1,T1=CME(star,pla)
    else:
        v,veff1,n1,T1=parker(pla,star)
    
    veff_07.append(veff1)
    Tcor_07.append(T1)
    n_07.append(n1)

    i+=1


test_results_SW_07=pd.DataFrame({'V_eff': veff_07, 'T_corona' : Tcor_07, 'n' : n_07},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_SW_07)  

                          V_eff      T_corona             n
Jupiter            1.046462e+06  2.615882e+06  3.340799e+06
Saturn             1.097192e+06  2.615882e+06  9.546269e+05
HD 209458b         5.455031e+05  2.000000e+06  7.553582e+10
OGLE-TR-56b        5.657283e+05  2.000000e+06  6.001124e+11
T-bootes (light)   5.499439e+05  2.000000e+06  5.891486e+10
T-bootes (medium)  5.499439e+05  2.000000e+06  5.891486e+10
T-bootes (heavy)   5.499439e+05  2.000000e+06  5.891486e+10


In [12]:
veff_CME=[]
Tcor_CME=[]
n_CME=[]

dua=1.49597870700e11 #m

i=0
for x in test_targets.itertuples():
    star=Star('Name', x.star_mass*1.989e30, x.star_radius, 1e9,1,1)
    pla=Planet(x.Index,x.planet_mass, x.planet_radius,x.planet_orbital_frequency, x.planet_orbital_frequency,1,x.planet_distance*dua)

    v,veff1,n1,T1=CME(star,pla)
    
    veff_CME.append(veff1)
    Tcor_CME.append(T1)
    n_CME.append(n1)

    i+=1


test_results_SW_CME=pd.DataFrame({'V_eff': veff_CME, 'T_corona' : Tcor_CME, 'n' : n_CME},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])

    
#print(test_results_dynamo)
print(test_results_SW_CME)  

                           V_eff   T_corona             n
Jupiter            526162.150520  2000000.0  5.133438e+04
Saturn             526088.762265  2000000.0  8.469636e+03
HD 209458b         545503.128080  2000000.0  7.553582e+10
OGLE-TR-56b        565728.298376  2000000.0  6.001124e+11
T-bootes (light)   549943.863979  2000000.0  5.891486e+10
T-bootes (medium)  549943.863979  2000000.0  5.891486e+10
T-bootes (heavy)   549943.863979  2000000.0  5.891486e+10


- Computing the radius of the magnetosphere for various stellar ages : 4.6, 1, 0.7 Gyr

In [13]:
Rs_46_sano1=[]
Rs_46_sano2=[]
Rs_46_Busse1=[]
Rs_46_Busse2=[]
Rs_46_MizuM1=[]
Rs_46_MizuM2=[]
Rs_46_MizuS1=[]
Rs_46_MizuS2=[]
Rs_46_RC=[]
Mj=1.56e27
for i in range (len(veff_1)):
    Rs_46_sano1.append(magnetosphere_radius(M_sano1[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_sano2.append(magnetosphere_radius(M_sano2[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_Busse1.append(magnetosphere_radius(M_Busse1[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_Busse2.append(magnetosphere_radius(M_Busse2[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_MizuM1.append(magnetosphere_radius(M_MizuM1[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_MizuM2.append(magnetosphere_radius(M_MizuM2[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_MizuS1.append(magnetosphere_radius(M_MizuS1[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_MizuS2.append(magnetosphere_radius(M_MizuS2[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])
    Rs_46_RC.append(magnetosphere_radius(M_RC[i]*Mj,n_46[i],veff_46[i],Tcor_46[i])/radius[i])

Rs_46=pd.DataFrame({'Rs4.6_sano1': Rs_46_sano1, 'Rs4.6_sano2': Rs_46_sano2, 'Rs4.6_Busse1': Rs_46_Busse1, 'Rs4.6_Busse2': Rs_46_Busse2,'Rs4.6_MizuM1': Rs_46_MizuM1,'Rs4.6_MizuM2': Rs_46_MizuM2,'Rs4.6_MizuS1': Rs_46_MizuS1,'Rs4.6_MizuS2': Rs_46_MizuS2,'Rs4.6_RC': Rs_46_RC},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
Rs_46.to_csv('Rs_46.csv', sep='\t')
print(Rs_46)



                   Rs4.6_sano1  Rs4.6_sano2  Rs4.6_Busse1  Rs4.6_Busse2  \
Jupiter              40.055528    40.055528     40.055528     40.055528   
Saturn               32.912859    36.264472     30.654178     34.283084   
HD 209458b            0.000000     0.971958      0.000000      0.923223   
OGLE-TR-56b           1.833788     1.945147      1.852270      1.982922   
T-bootes (light)      3.201880     3.074382      3.379069      3.221608   
T-bootes (medium)     3.056996     2.965993      3.355409      3.238803   
T-bootes (heavy)      3.466386     3.315127      3.798406      3.604207   

                   Rs4.6_MizuM1  Rs4.6_MizuM2  Rs4.6_MizuS1  Rs4.6_MizuS2  \
Jupiter               40.055528     40.055528     40.055528     40.055528   
Saturn                32.912859     36.264472     35.337966     38.360373   
HD 209458b             0.000000      1.159798      0.000000      1.456995   
OGLE-TR-56b            2.006284      2.128119      2.173105      2.283948   
T-bootes (ligh

In [14]:
Rs_1_sano1=[]
Rs_1_sano2=[]
Rs_1_Busse1=[]
Rs_1_Busse2=[]
Rs_1_MizuM1=[]
Rs_1_MizuM2=[]
Rs_1_MizuS1=[]
Rs_1_MizuS2=[]
Rs_1_RC=[]
Mj=1.56e27
for i in range (len(veff_1)):
    Rs_1_sano1.append(magnetosphere_radius(M_sano1[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_sano2.append(magnetosphere_radius(M_sano2[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_Busse1.append(magnetosphere_radius(M_Busse1[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_Busse2.append(magnetosphere_radius(M_Busse2[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_MizuM1.append(magnetosphere_radius(M_MizuM1[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_MizuM2.append(magnetosphere_radius(M_MizuM2[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_MizuS1.append(magnetosphere_radius(M_MizuS1[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_MizuS2.append(magnetosphere_radius(M_MizuS2[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])
    Rs_1_RC.append(magnetosphere_radius(M_RC[i]*Mj,n_1[i],veff_1[i],Tcor_1[i])/radius[i])

Rs_1=pd.DataFrame({'Rs1_sano1': Rs_1_sano1, 'Rs1_sano2': Rs_1_sano2, 'Rs1_Busse1': Rs_1_Busse1, 'Rs1_Busse2': Rs_1_Busse2,'Rs1_MizuM1': Rs_1_MizuM1,'Rs1_MizuM2': Rs_1_MizuM2,'Rs1_MizuS1': Rs_1_MizuS1,'Rs1_MizuS2': Rs_1_MizuS2,'Rs1_RC': Rs_1_RC},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
Rs_1.to_csv('Rs_1.csv', sep='\t')
print(Rs_1)

                   Rs1_sano1  Rs1_sano2  Rs1_Busse1  Rs1_Busse2  Rs1_MizuM1  \
Jupiter            22.669699  22.669699   22.669699   22.669699   22.669699   
Saturn             18.650833  20.550101   17.370899   19.427303   18.650833   
HD 209458b          0.000000   0.971958    0.000000    0.923223    0.000000   
OGLE-TR-56b         1.833788   1.945147    1.852270    1.982922    2.006284   
T-bootes (light)    3.201880   3.074382    3.379069    3.221608    3.820673   
T-bootes (medium)   3.056996   2.965993    3.355409    3.238803    3.647789   
T-bootes (heavy)    3.466386   3.315127    3.798406    3.604207    4.136298   

                   Rs1_MizuM2  Rs1_MizuS1  Rs1_MizuS2     Rs1_RC  
Jupiter             22.669699   22.669699   22.669699  22.669699  
Saturn              20.550101   20.025076   21.737792  14.372939  
HD 209458b           1.159798    0.000000    1.456995   3.479067  
OGLE-TR-56b          2.128119    2.173105    2.283948   3.228340  
T-bootes (light)     3.668535   

In [15]:
Rs_07_sano1=[]
Rs_07_sano2=[]
Rs_07_Busse1=[]
Rs_07_Busse2=[]
Rs_07_MizuM1=[]
Rs_07_MizuM2=[]
Rs_07_MizuS1=[]
Rs_07_MizuS2=[]
Rs_07_RC=[]
Mj=1.56e27
for i in range (len(veff_1)):
    Rs_07_sano1.append(magnetosphere_radius(M_sano1[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_sano2.append(magnetosphere_radius(M_sano2[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_Busse1.append(magnetosphere_radius(M_Busse1[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_Busse2.append(magnetosphere_radius(M_Busse2[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_MizuM1.append(magnetosphere_radius(M_MizuM1[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_MizuM2.append(magnetosphere_radius(M_MizuM2[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_MizuS1.append(magnetosphere_radius(M_MizuS1[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_MizuS2.append(magnetosphere_radius(M_MizuS2[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])
    Rs_07_RC.append(magnetosphere_radius(M_RC[i]*Mj,n_07[i],veff_07[i],Tcor_07[i])/radius[i])

Rs_07=pd.DataFrame({'Rs0.7_sano1': Rs_07_sano1, 'Rs0.7_sano2': Rs_07_sano2, 'Rs0.7_Busse1': Rs_07_Busse1, 'Rs0.7_Busse2': Rs_07_Busse2,'Rs0.7_MizuM1': Rs_07_MizuM1,'Rs0.7_MizuM2': Rs_07_MizuM2,'Rs0.7_MizuS1': Rs_07_MizuS1,'Rs0.7_MizuS2': Rs_07_MizuS2,'Rs0.7_RC': Rs_07_RC},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
Rs_07.to_csv('Rs_07.csv', sep='\t')
print(Rs_07)

                   Rs0.7_sano1  Rs0.7_sano2  Rs0.7_Busse1  Rs0.7_Busse2  \
Jupiter              19.883649    19.883649     19.883649     19.883649   
Saturn               16.362794    18.029066     15.239880     17.044009   
HD 209458b            0.000000     0.971958      0.000000      0.923223   
OGLE-TR-56b           1.833788     1.945147      1.852270      1.982922   
T-bootes (light)      3.201880     3.074382      3.379069      3.221608   
T-bootes (medium)     3.056996     2.965993      3.355409      3.238803   
T-bootes (heavy)      3.466386     3.315127      3.798406      3.604207   

                   Rs0.7_MizuM1  Rs0.7_MizuM2  Rs0.7_MizuS1  Rs0.7_MizuS2  \
Jupiter               19.883649     19.883649     19.883649     19.883649   
Saturn                16.362794     18.029066     17.568449     19.071054   
HD 209458b             0.000000      1.159798      0.000000      1.456995   
OGLE-TR-56b            2.006284      2.128119      2.173105      2.283948   
T-bootes (ligh

In [16]:
Rs_CME_sano1=[]
Rs_CME_sano2=[]
Rs_CME_Busse1=[]
Rs_CME_Busse2=[]
Rs_CME_MizuM1=[]
Rs_CME_MizuM2=[]
Rs_CME_MizuS1=[]
Rs_CME_MizuS2=[]
Rs_CME_RC=[]
Mj=1.56e27
for i in range (len(veff_1)):
    Rs_CME_sano1.append(magnetosphere_radius(M_sano1[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_sano2.append(magnetosphere_radius(M_sano2[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_Busse1.append(magnetosphere_radius(M_Busse1[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_Busse2.append(magnetosphere_radius(M_Busse2[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_MizuM1.append(magnetosphere_radius(M_MizuM1[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_MizuM2.append(magnetosphere_radius(M_MizuM2[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_MizuS1.append(magnetosphere_radius(M_MizuS1[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_MizuS2.append(magnetosphere_radius(M_MizuS2[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])
    Rs_CME_RC.append(magnetosphere_radius(M_RC[i]*Mj,n_CME[i],veff_CME[i],Tcor_CME[i])/radius[i])

Rs_CME=pd.DataFrame({'RsCME_sano1': Rs_CME_sano1, 'RsCME_sano2': Rs_CME_sano2, 'RsCME_Busse1': Rs_CME_Busse1, 'RsCME_Busse2': Rs_CME_Busse2,'RsCME_MizuM1': Rs_CME_MizuM1,'RsCME_MizuM2': Rs_CME_MizuM2,'RsCME_MizuS1': Rs_CME_MizuS1,'RsCME_MizuS2': Rs_CME_MizuS2,'RsCME_RC': Rs_CME_RC},index=['Jupiter','Saturn','HD 209458b','OGLE-TR-56b','T-bootes (light)','T-bootes (medium)','T-bootes (heavy)'])
Rs_CME.to_csv('Rs_CME.csv', sep='\t')
print(Rs_CME)

                   RsCME_sano1  RsCME_sano2  RsCME_Busse1  RsCME_Busse2  \
Jupiter              49.531228    49.531228     49.531228     49.531228   
Saturn               45.354023    49.972556     42.241553     47.242198   
HD 209458b            0.000000     0.971958      0.000000      0.923223   
OGLE-TR-56b           1.833788     1.945147      1.852270      1.982922   
T-bootes (light)      3.201880     3.074382      3.379069      3.221608   
T-bootes (medium)     3.056996     2.965993      3.355409      3.238803   
T-bootes (heavy)      3.466386     3.315127      3.798406      3.604207   

                   RsCME_MizuM1  RsCME_MizuM2  RsCME_MizuS1  RsCME_MizuS2  \
Jupiter               49.531228     49.531228     49.531228     49.531228   
Saturn                45.354023     49.972556     48.695828     52.860714   
HD 209458b             0.000000      1.159798      0.000000      1.456995   
OGLE-TR-56b            2.006284      2.128119      2.173105      2.283948   
T-bootes (ligh